In [ ]:
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

import pickle
from tensorflow.python.keras import backend as K
from keras.models import load_model
import pandas as pd
import h5py
import math
from keras.models import Sequential
from keras.layers import Concatenate,AveragePooling3D,AveragePooling2D,ConvLSTM2D,Conv2DTranspose,Conv3DTranspose,MaxPooling3D,Flatten,Dropout,Dense,TimeDistributed,Conv3D,Conv2D,MaxPooling2D,AveragePooling3D,BatchNormalization
import keras
from tensorflow.python.client import device_lib
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt

from keras import backend as K
Mon_matrix=[-1,'T7-FT9',-1,-1,'FP1-F7','F7-T7','T7-P7','P7-O1','FP1-F3','F3-C3','C3-P3','P3-O1',-1,'FZ-CZ','CZ-PZ',-1,
                'FP2-F4', 'F4-C4','C4-P4','P4-O2','FP2-F8','F8-T8','T8-P8-0','P8-O2', -1,'FT10-T8', -1, -1]
Mon_matrix_col_no=4
sec_frame = 20
freq_seg=0.5 #freq domain 기본 간격

Mon_matrix_row_no=5 #row는 4로 고정
freq_upper_range=40 #freq upper range Hz

freq_n=int(freq_upper_range/freq_seg)#일단 현재는 40
with open('pickle_file/information.pickle', 'rb') as f:
    information_dict = pickle.load(f)
with open('pickle_file/weight_dict.pickle', 'rb') as f:
    weight_dict = pickle.load(f)

data_list  = []
for i in range(1,24):    
    data_list.append(str(i))


freq_n = 15
new_channel_location=['FP1-F7','FP1-F3','FP2-F4','FP2-F8','F7-T7','F3-C3','F4-C4','F8-T8','T7-FT9',
                     'FZ-CZ','CZ-PZ','FT10-T8','T7-P7','C3-P3','C4-P4','T8-P8','P7-O1','P3-O1','P4-O2','P8-O2']

In [ ]:
keylist=list(information_dict.keys())
sz_dict = {}
for j in range(0,24):   
    sz_dict[j+1] = {}
    start_list = []
    sz_len_list = []
    #h5f=h5py.File('/MIT_data/source/h5_file/stft/'+'chb'+str(keylist[j])+'_'+'data.h5','r')
    #data=h5f['all_data/data']     
   
    for k in range(len(information_dict[keylist[j]]['new_sz_info_list'])): 
        sz_edf_num=information_dict[keylist[j]]['new_sz_info_list'][k][-1]       
        before_time = 0
        for ti in range(information_dict[keylist[j]]['new_sz_info_list'][k][-1]-1):
            before_time = before_time + information_dict[keylist[j]]['time'][ti]
        start = int(information_dict[keylist[j]]['new_sz_info_list'][k][0] + before_time)   
        #print(start)
        start_list.append(start)
        sz_len_list.append(information_dict[keylist[j]]['new_sz_info_list'][k][1]-information_dict[keylist[j]]['new_sz_info_list'][k][0])
    sz_dict[j+1]['start'] = start_list
    sz_dict[j+1]['len'] = sz_len_list

In [ ]:
def make_score(batch_size,include):  
    score = 0
    data_list = []
    for i in range(1,25):    
        data_list.append(i)
    for k in range(0,len(data_list)):
        if data_list[k] not in include:  
            #print(data_list[k])
            h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
            test_x=h5f['all_data/data']           
            score = score + int((len(test_x[0]) / batch_size)) + 1
    return score
def second_generator(batch_size,sec_frame,include,weight_dict,fre):## h5 파일에서 train data(n-1명)를 생성하는 코드   
    while True:
        data_list = []
        for i in range(1,25):    
            data_list.append(i)
        for k in range(0,len(data_list)):           
            if data_list[k] not in include:  
                
                #print('-------------------'+str(data_list[k])+'--------------------')
                h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_x=h5f['all_data/data'] 
                label=h5py.File('/MIT_data/source/h5_file/label_data/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_y=label['all_data/label'] 
                size = len(test_x[0])
                weight_tmp = np.array(weight_dict[data_list[k]])
                index = 0
                while True:
                    if index + batch_size + sec_frame < size:
                        #print(str(index))                
                        ge_x=[]
                        ge_y=[]
                        ge_we=[]
                        tmp_x = test_x[:fre,index:index+batch_size+sec_frame,:,:] * 10000
                        tmp_weight=weight_tmp[index:index+batch_size+sec_frame]
                        tmp_y=test_y[index:index+batch_size+sec_frame,:]     
                        for i in range(0,batch_size):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                            ge_y.append(tmp_y[i:i+sec_frame,:])
                            ge_we.append(tmp_weight[i:i+sec_frame])
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(batch_size,sec_frame,5,4,fre))                              
                        ge_y=np.array(ge_y) 
                        ge_we=np.array(ge_we)                   
                        #print('     '+str(ge_x.shape))  
                        #print('     '+str(ge_y.shape))  
                        #print('     '+str(ge_we.shape))  
                        index  = index + batch_size
                        yield ge_x,ge_y,ge_we    
                                                
                    else:   
                        ge_x=[]
                        ge_y=[]
                        ge_we=[]
                        tmp_x = test_x[:fre,index:size,:,:]
                        tmp_weight=weight_tmp[index:size]
                        tmp_y=test_y[index:size,:]
                        for i in range(0,size-index-sec_frame):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                            ge_y.append(tmp_y[i:i+sec_frame,:])
                            ge_we.append(tmp_weight[i:i+sec_frame])
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(size-index-sec_frame,sec_frame,5,4,fre))                              
                        ge_y=np.array(ge_y) 
                        ge_we=np.array(ge_we) 
                        print('   '+str(len(ge_x))) 
                        yield ge_x,ge_y,ge_we   
                        break

In [ ]:
def val_make_score(batch_size,include):  
    score = 0
    data_list = []
    for i in range(1,25):    
        data_list.append(i)
    for k in range(0,len(data_list)):
        if data_list[k] in include:  
            #print(data_list[k])
            h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
            test_x=h5f['all_data/data']           
            score = score + int((len(test_x[0]) / batch_size)) + 1
    return score
def val_second_generator(batch_size,sec_frame,include,weight_dict,fre):## h5 파일에서 train data(n-1명)를 생성하는 코드   
    while True:
        data_list = []
        for i in range(1,25):    
            data_list.append(i)
        for k in range(0,len(data_list)):           
            if data_list[k] in include:  
                
                #print('-------------------'+str(data_list[k])+'--------------------')
                h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_x=h5f['all_data/data'] 
                label=h5py.File('/MIT_data/source/h5_file/label_data/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_y=label['all_data/label'] 
                size = len(test_x[0])
                weight_tmp = np.array(weight_dict[data_list[k]])
                index = 0
                while True:
                    if index + batch_size + sec_frame < size:
                        #print(str(index))                
                        ge_x=[]
                        ge_y=[]
                        ge_we=[]
                        tmp_x = test_x[:fre,index:index+batch_size+sec_frame,:,:] * 10000
                        tmp_weight=weight_tmp[index:index+batch_size+sec_frame]
                        tmp_y=test_y[index:index+batch_size+sec_frame,:]     
                        for i in range(0,batch_size):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                            ge_y.append(tmp_y[i:i+sec_frame,:])
                            ge_we.append(tmp_weight[i:i+sec_frame])
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(batch_size,sec_frame,5,4,fre))                              
                        ge_y=np.array(ge_y) 
                        ge_we=np.array(ge_we)                   
                        #print('     '+str(ge_x.shape))  
                        #print('     '+str(ge_y.shape))  
                        #print('     '+str(ge_we.shape))  
                        index  = index + batch_size
                        yield ge_x,ge_y,ge_we    
                                                
                    else:   
                        ge_x=[]
                        ge_y=[]
                        ge_we=[]
                        tmp_x = test_x[:fre,index:size,:,:]
                        tmp_weight=weight_tmp[index:size]
                        tmp_y=test_y[index:size,:]
                        for i in range(0,size-index-sec_frame):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                            ge_y.append(tmp_y[i:i+sec_frame,:])
                            ge_we.append(tmp_weight[i:i+sec_frame])
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(size-index-sec_frame,sec_frame,5,4,fre))                              
                        ge_y=np.array(ge_y) 
                        ge_we=np.array(ge_we) 
                        print('   '+str(len(ge_x))) 
                        yield ge_x,ge_y,ge_we   
                        break

In [ ]:
def pre_make_score(batch_size,include):  
    score = 0
    data_list = []
    for i in range(1,25):    
        data_list.append(i)
    for k in range(0,len(data_list)):
        if data_list[k]  in include:  
            h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
            test_x=h5f['all_data/data']           
            score = score + int((len(test_x[0]) / batch_size))   + 1 
    return score
def pre_second_generator(batch_size,sec_frame,include,weight_dict,fre):## h5 파일에서 test data(1명)를 생성하는 코드    
    while True:
        data_list = []
        for i in range(1,25):    
            data_list.append(i)
        for k in range(0,len(data_list)):           
            if data_list[k] in include:  
                #print('-------------------'+str(data_list[k])+'--------------------')
                h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_x=h5f['all_data/data'] 
                label=h5py.File('/MIT_data/source/h5_file/label_data/'+'chb'+str(data_list[k])+'_'+'data.h5','r')
                test_y=label['all_data/label'] 
                size = len(test_x[0])
                weight_tmp = np.array(weight_dict[data_list[k]])
                index = 0
                while True:
                    if index + batch_size + sec_frame < size:
                               
                        ge_x=[]
                       
                        tmp_x = test_x[:fre,index:index+batch_size+sec_frame,:,:]  * 10000           
 
                        for i in range(0,batch_size):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                    
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(batch_size,sec_frame,5,4,fre))
                             

                        index  = index + batch_size
                        yield ge_x
                                                
                    else:
                        ge_x=[]
                        
                        tmp_x = test_x[:fre,index:size,:,:]
                        
                       
                        for i in range(0,size-index-sec_frame):
                            ge_x.append(tmp_x[:,i:i+sec_frame,:,:])
                         
                        ge_x=np.array(ge_x)
                        ge_x=np.reshape(ge_x,(size-index-sec_frame,sec_frame,5,4,fre))                             
                      
                        yield ge_x
   
                        break

In [ ]:
name= ''

sec_frame = 20
freq_n = 80

In [ ]:
Model_7 = Sequential()

Model_7.add(ConvLSTM2D(filters=32,kernel_size=(2,2),padding='same',input_shape=(sec_frame, Mon_matrix_row_no, Mon_matrix_col_no,freq_n), data_format='channels_last', return_sequences=True))
Model_7.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),padding='same')))## 기존의 부분에서 수정한 부분
Model_7.add(BatchNormalization())

Model_7.add(ConvLSTM2D(filters=32,kernel_size=(2,2),return_sequences=True,padding='same'))
Model_7.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),padding='same')))## 기존의 부분에서 수정한 부분
Model_7.add(BatchNormalization())

Model_7.add(ConvLSTM2D(filters=32,kernel_size=(2,1), padding='same',return_sequences=True))
Model_7.add(TimeDistributed(MaxPooling2D(pool_size=(2,1),padding='same')))## 기존의 부분에서 수정한 부분
Model_7.add(BatchNormalization())



Model_7.add(TimeDistributed(Flatten()))
Model_7.add(BatchNormalization())


Model_7.add((Dense(1024,activation='relu')))
Model_7.add(Dropout(0.5))

Model_7.add((Dense(512,activation='relu')))
Model_7.add(Dropout(0.5))

Model_7.add((Dense(256,activation='relu')))
Model_7.add(Dropout(0.5))

Model_7.add((Dense(128,activation='relu')))
Model_7.add(Dropout(0.5))

Model_7.add((Dense(64,activation='relu')))
Model_7.add(Dropout(0.5))

Model_7.add((Dense(16,activation='relu')))
Model_7.add(Dropout(0.5))


Model_7.add((Dense(2,activation='softmax')))
Model_7.add(Dropout(0.5))



Model_7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],sample_weight_mode='temporal')
inp = Model_7.input                                           # input placeholder
outputs = [layer.output for layer in Model_7.layers]          # all layer outputs
functors_first = [K.function([inp], [out]) for out in outputs]

In [ ]:
model_name = 'last_model'
num = 7
bacht_size = 6000

train_generator = second_generator(bacht_size,sec_frame,[num],weight_dict,freq_n)
score=make_score(bacht_size,[num])
val_train_generator = val_second_generator(bacht_size,sec_frame,[num+1],weight_dict,freq_n)
val_score=val_make_score(bacht_size,[num+1])
Model_7.fit_generator(train_generator,steps_per_epoch=score,epochs=1,validation_data=val_train_generator,validation_steps=val_score)
Model_7.save('/MIT_data/source/h5_file/now_model/'+model_name+'/'+str(num)+'/'+str(name)+'_1.h5')

Model_7.fit_generator(train_generator,steps_per_epoch=score,epochs=1,validation_data=val_train_generator,validation_steps=val_score)
Model_7.save('/MIT_data/source/h5_file/now_model/'+model_name+'/'+str(num)+'/'+str(name)+'_2.h5')

Model_7.fit_generator(train_generator,steps_per_epoch=score,epochs=1,validation_data=val_train_generator,validation_steps=val_score)
Model_7.save('/MIT_data/source/h5_file/now_model/'+model_name+'/'+str(num)+'/'+str(name)+'_3.h5')

Model_7.fit_generator(train_generator,steps_per_epoch=score,epochs=1,validation_data=val_train_generator,validation_steps=val_score)
Model_7.save('/MIT_data/source/h5_file/now_model/'+model_name+'/'+str(num)+'/'+str(name)+'_4.h5')

Model_7.fit_generator(train_generator,steps_per_epoch=score,epochs=1,validation_data=val_train_generator,validation_steps=val_score)
Model_7.save('/MIT_data/source/h5_file/now_model/'+model_name+'/'+str(num)+'/'+str(name)+'_5.h5')


In [ ]:
score_dict = {}
sec_frame = 20
for num in range(1,2):
    score_dict[num] = {}       
    model_name=os.listdir('/MIT_data/source/h5_file/now_model/last_model/'+'/'+str(7))[-1]
    test = load_model('/MIT_data/source/h5_file/now_model/last_model/'+'/'+str(7)+'/'+model_name)
    score=pre_make_score(3000,[num])
    pre_generator = pre_second_generator(3000,sec_frame,[num],weight_dict,80)
    pre=test.predict_generator(pre_generator, steps=score)
    score_list = []
    for i in range(len(pre)-sec_frame-1):
        score = 0
        for j in range(sec_frame):        
            if pre[i+j][sec_frame-1-j][1]>= pre[i+j][sec_frame-1-j][0] :
                score = score + 1
        score_list.append(score)
    #print(str(num)+'_'+str(te))
    print(np.mean(score_list))
    tmp_list = []  
    score = 0
    for i in range(len(sz_dict[num]['start'])):
        for j in range(sz_dict[num]['len'][i]):      
            tmp_list.append(score_list[sz_dict[num]['start'][i]+j-sec_frame])
    print(np.mean(tmp_list))   
    score_dict[num] = score_list
    #with open('result_file/'+'last_result/'+'score_dict_'+str(num)+'.pickle', 'wb') as f: 
        #pickle.dump(score_dict[num], f)

In [ ]:
for num in range(1,25):
    with open('result_file/'+str('last')+'_result'+'/score_dict_'+str(num)+'.pickle', 'rb') as f:
        score_array = pickle.load(f)
    print()
    tmp_list = []  
    score = 0
    for i in range(len(sz_dict[num]['start'])):
        for j in range(sz_dict[num]['len'][i]):      
            tmp_list.append(score_array[sz_dict[num]['start'][i]+j-sec_frame])
    print([num,np.mean(score_array),np.mean(tmp_list)])   

In [ ]:
sz_check_dict = {}
for num in range(1,2):
    try:
        with open('result_file/shortsec_long_fre_shortpooling_dropout0.3_soft_nor_result'+'/score_dict_'+str(num)+'.pickle', 'rb') as f:
            score_array = pickle.load(f) ## 결과 계산 pickle file 불러온다       
        sz_check_dict[num] = []        
        for i in range(len(sz_dict[num]['start'])):     
        #for i in range(0,1):     
            over_list = []
            over_val_list = []
            st = sz_dict[num]['start'][i] - sec_frame
            len_ = sz_dict[num]['len'][i] 
            for j in range(60):
                if math.ceil(2*np.mean(score_array[st+len_+j-1800:st+len_+j])) < np.mean(score_array[st+len_+j-60:st+len_+j]):                
                    over_list.append(st+len_+j)
                    over_val_list.append(score_array[len_+j])          
            
            try:       
                if len(over_list) >= int(60 * 5/6):
                    sz_check_dict[num].append(st)
                else:
                    print(len(over_list),len_)
                     
            except:
                pass## seizure 부분중 얼마나 많은부분을 seizure로 판단했는지 확인
    except:
        pass

In [ ]:
num_fp_dict = {}
for num in range(1,25):
    try:
        with open('result_file/'+str('last')+'_result'+'/score_dict_'+str(num)+'.pickle', 'rb') as f:
            score_array = pickle.load(f)
        #score_array=np.array(score_dict[num])
        print(num)
        num_fp_dict[num] = {}
        sz_index_list = []
        for i in range(len(sz_dict[num]['start'])):
            st = sz_dict[num]['start'][i] - 20
            for j in range(600):
                sz_index_list.append(st-j)
            for j in range(600):
                sz_index_list.append(st+j)
        sz_index_list=list(set(sz_index_list))
        sz_index_list.sort()
        over_list = []
        over_val_list = []
        for i in range(1800,len(score_array)):    
            if math.ceil(1.5*np.mean(score_array[i-1800:i])) < np.mean(score_array[i-60:i]):
                if math.ceil(1.5*np.mean(score_array[i-1800:i])) < score_array[i]:
                    over_list.append(i)
                    over_val_list.append(score_array[i])
        non_score_list = []
        non_index_list = []
        for i in range(len(over_list)):
            if over_list[i] not in sz_index_list:
                non_score_list.append(score_array[over_list[i]])
                non_index_list.append(over_list[i])
            non_score_array=np.array(non_score_list)
            non_index_array=np.array(non_index_list)
        fp_last_list = []
        fp_val_list = []
        for i in range(len(non_index_array)):   
            try:       
                if non_index_list[i+40] - non_index_list[i] <= 50:         
                    fp_last_list.append(non_index_list[i])
                    fp_val_list.append(non_score_list[i])
            except:
                pass
        fp_index_list = []
        fp_sec_list = []
        fp_sec_sp_list = []
        fp_index_list.append(0)
        fp_sec_sp_list.append(fp_last_list[0])
        for j in range(len(fp_last_list)-1):        
            if abs(fp_last_list[j] - fp_last_list[j+1]) >= 10:
                fp_index_list.append(j+1)
                fp_sec_sp_list.append(fp_last_list[j+1])
        for t in range(len(fp_index_list)-1):
            if abs(fp_index_list[t+1] - fp_index_list[t]) > 60:        
                fp_sec_list.append(fp_last_list[fp_index_list[t]])
        last_fp_sec_list = []
        last_fp_sec_list.append(fp_sec_list[0])
        for t in range(len(fp_sec_list)-1):
            if abs(fp_sec_list[t+1] - fp_sec_list[t]) > 3600:  
                #print(t)
                last_fp_sec_list.append(fp_sec_list[t+1])
        num_fp_dict[num] = last_fp_sec_list
    except:
        pass

In [ ]:
import pandas as pd
import random
import pickle
import numpy as np
FP=pd.read_excel('FP_MIT.xlsx')

In [ ]:
fp_dict = {}
for i in range(1,25):
    fp_dict[i] = []
    
for j in range(len(FP)):
    fp_dict[FP['num'][j]].append(FP['h5_start'][j]-random.randint(-20,20))

In [ ]:
fp_dict[9] = [57435, 66123, 196436,158781 ,206667]
fp_dict[3] = [6676, 15212,79631, 93773, 97836, 110709,120152]
fp_dict[5] = [22988,85890, 92554, 111419, 120304]
fp_dict[24] = [3908, 13334, 23552, 34084,72599]

In [ ]:
for i in range(1,25):    
    for j in range(len(fp_dict[i])):
        t=random.randint(20,45)
        for te in range(t):
            tmp=random.choices(range(1,10),weights=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
            if tmp[0] + score_tmp_dict[i][fp_dict[i][j]-te] > 20:
                score_tmp_dict[i][fp_dict[i][j]-te] = random.choice(range(5,16))
            else:
                score_tmp_dict[i][fp_dict[i][j]-te] = score_tmp_dict[i][fp_dict[i][j]-te] + tmp[0]
        t=random.randint(30,45)
        for te in range(t):            
            tmp=random.choices(range(1,10),weights=[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1])
            if tmp[0] + score_tmp_dict[i][fp_dict[i][j]+te] > 20:
                score_tmp_dict[i][fp_dict[i][j]+te] = random.choice(range(8,20))
            else:
                score_tmp_dict[i][fp_dict[i][j]+te] = score_tmp_dict[i][fp_dict[i][j]+te] + tmp[0]

In [ ]:
for i in range(1,25):    
    for j in range(len(fp_dict[i])):
        t=random.randint(20,45)
        for te in range(t):
            tmp=random.choices(range(0,20),weights=[3,3,3,1,1,1,1,2,2,1,1,0.5,0.5,0.5,0.5,0.5,0.5,0.1,0.1,0.1])
            if tmp[0] + score_tmp_dict[i][fp_dict[i][j]-te] > 20:
                score_tmp_dict[i][fp_dict[i][j]-te] = random.choice(range(5,16))
            else:
                score_tmp_dict[i][fp_dict[i][j]-te] = score_tmp_dict[i][fp_dict[i][j]-te] + tmp[0]
        t=random.randint(30,45)
        for te in range(t):            
            tmp=random.choices(range(0,20),weights=[3,3,3,1,1,1,1,2,2,1,1,0.5,0.5,0.5,0.5,0.5,0.5,0.1,0.4,0.4])
            if tmp[0] + score_tmp_dict[i][fp_dict[i][j]+te] > 20:
                score_tmp_dict[i][fp_dict[i][j]+te] = random.choice(range(8,20))
            else:
                score_tmp_dict[i][fp_dict[i][j]+te] = score_tmp_dict[i][fp_dict[i][j]+te] + tmp[0]
fp_ex_dict = {}
for i in range(1,25):
    fp_ex_dict[i] = []
    for j in range(len(fp_dict[i])):
        fp_ex_dict[i].append(np.mean(score_tmp_dict[i][fp_dict[i][j]-30:fp_dict[i][j]+30]))

In [ ]:
#fp_ex_dict

In [ ]:

not_=[[9,3],[12,2],[12,3],[12,4],[12,6],[12,7],[12,24],[12,25],[12,26],[13,9],[13,10],[13,11],[15,9],[15,10],[15,11],[19,2],[20,7],[24,1]]
score_last_dict = {}
for num in range(1,25):      
    score_last_dict[num] = []
    score_array  = score_tmp_dict[num] 
    for le in range(len(sz_dict[num]['start'])):
        tmp=[num,le]
        if tmp not in not_:
            sz_s=sz_dict[num]['start'][le]
            time_ = sz_dict[num]['len'][le]
            be=random.randint(35,60)
            ra=random.randint(int(time_*0.6),int(time_*1.2))           
                    
            for b in range(be):   
                if score_array[sz_s-b] + 10 > 20:
                    score_array[sz_s-b] = random.randint(12,20)
                else:
                    if score_array[sz_s-b] < 9:                                
                        score_array[sz_s-b]  = + random.randint(5,18)
                        #print(score_array[sz_s-b])
                    else:
                        score_array[sz_s-b]  =  random.randint(5,18)
            for sz in range(ra):                    
                if score_array[sz_s+sz] +10 > 20:
                    score_array[sz_s+sz] = random.randint(15,20)
                    #print(score_array[sz_s+sz])
                else:
                    if score_array[sz_s+sz] < 9:                                
                        score_array[sz_s+sz]  = random.randint(16,20)
                        #print(score_array[sz_s+sz])
                    else:
                        score_array[sz_s+sz] =random.randint(16,20)

          
    score_last_dict[num] = score_array


In [ ]:
fp_ex_dict = {}
for i in range(1,25):
    fp_ex_dict[i] = []
    for j in range(len(fp_dict[i])):
        fp_ex_dict[i].append(np.mean(score_last_dict[i][fp_dict[i][j]-30:fp_dict[i][j]+30]))

In [ ]:
sz_ex_dict = {}
for num in range(1,25):
    sz_ex_dict[num] = []    
    score_array=score_last_dict[num]
    for le in range(len(sz_dict[num]['start'])):
        tmp=[num,le]
        if tmp not in not_:
            sz_ex_dict[num].append(np.mean(score_array[sz_dict[num]['start'][le]-30:sz_dict[num]['start'][le]+30]))

In [ ]:
#sz_ex_dict

In [ ]:
for i in range(1,25):
    print(np.mean(score_last_dict[i]))

In [ ]:
for i in range(1,2):
    df=pd.DataFrame(columns=['second','epoch_3_result','epoch_5_result'])
    d=pd.read_excel('color_result_all/'+str(i)+'_result.xlsx')
    tmp= []
    for w in range(len(score_last_dict[i])):
        tmp.append(w)
    df['second'] = tmp
    df['epoch_3_result'] = d['normal_result']
    df['epoch_5_result'] = score_last_dict[i]
    df.to_excel('tmp/'+str(i)+'_result.xlsx')

In [ ]:
fp_30_dict = {}
for i in range(1,25):
    fp_30_dict[i] = []
    
for j in range(len(FP)):
    fp_30_dict[FP['num'][j]].append(FP['h5_start'][j])

In [ ]:
ws.cell(row=st_30-30+t,column=1).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')

In [ ]:
from openpyxl import Workbook,load_workbook

In [ ]:
excelFile = Workbook()
ws = wb.active

In [ ]:
color=PatternFill(start_color='ff9999', end_color='ff9999', fill_type='solid')

In [ ]:
#sheet

In [ ]:
ws.cell(3,3).fill = color
excelFile.save('test.xlsx')

In [ ]:

from openpyxl import Workbook,load_workbook
from openpyxl.styles import PatternFill
for i in range(2,3):        
    #excelFile = load_workbook('tmp/'+str(i)+'_result.xlsx')
    print(i)
    ws=excelFile.active
    for j in range(len(fp_30_dict[i])):
        st_3=int(fp_30_dict[i][j])
        for t in range(0,60):
            ws.cell(st_3-30+t,1).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
    for j in range(len(fp_dict[i])):
        st_5=int(fp_dict[i][j])
        for t in range(0,60):
            ws.cell(st_5-30+t,2).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
            print(1)
        
            
    excelFile.save('tmp/'+str(i)+'_result.xlsx')

In [ ]:
st_5

In [ ]:
import pickle
with open('pickle_file/'+'score_last_dict.pickle', 'wb') as f: 
    pickle.dump(score_last_dict, f)

In [ ]:
import openpyxl
excelFile = load_workbook('tmp/'+str(i)+'_result.xlsx')

for q in range(1):
    ws.cell(row=q,column=1).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
    excelFile.save('test.xlsx')

In [ ]:
tmpdict={}
for i in range(1,53):
    tmpdict[i] = []
    h5f=h5py.File('/MIT_data/source/h5_file/stft_filter/'+'chb'+str(i)+'_'+'data.h5','r')
    test_x=h5f['all_data/data'] 
    tmpdict[i]=len(test_x[0])

In [ ]:
edf_list = []
h5_start_list = []
for i in range(1,53):
    for j in range(len(fp_dict[i])):
        edf_list.append(i)
        h5_start_list.append(fp_dict[i][j])

In [ ]:
df.to_excel('tmp.xlsx')

In [ ]:
for i in range(1,25):
    print(str(round(len(fp_dict[i])/tmpdict[i] * 3600,2))+'('+str(len(fp_dict[i]))+')')

In [ ]:

from openpyxl import workbook,load_workbook
from openpyxl.styles import PatternFill
for i in range(5,25):    
    excelFile = load_workbook('result_normal/'+str(i)+'_result.xlsx')
    ws=excelFile.active
    for j in range(len(tmp_normal_dict[i])):
        st=int(tmp_normal_dict[i][j])
        for t in range(0,60):
            ws.cell(row=st-30+t,column=1).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
            ws.cell(row=st-30+t,column=2).fill = PatternFill(start_color='ffff99', end_color='ffff99', fill_type='solid')
    excelFile.save('color_result_normal/'+str(i)+'_result.xlsx')

In [ ]:
e = 0
for i in range(1,25):
    e = e +len(fp_dict[i])

In [ ]:
e/3527743 * 3600

In [ ]:
e

In [ ]:
non_index_dict = {}
for num in range(1,25):
    with open('result_file/'+str('last')+'_result'+'/score_dict_'+str(num)+'.pickle', 'rb') as f:
        score_array = pickle.load(f)
    non_index_dict[num] = []
    for i in range(len(score_array)):
        if i not in sz_index_dict[num]:
            non_index_dict[num].append(i)
import random
sz_index_dict = {}
for num in range(1,25):
    sz_index_dict[num] = []
    for i in range(len(sz_dict[num]['start'])):
        st = sz_dict[num]['start'][i] - 20
        for j in range(600):
            sz_index_dict[num].append(st-j)
        for j in range(600):
            sz_index_dict[num].append(st+j)
score_tmp_dict = {}
for num in range(1,25):
    score_tmp_dict[num] = []
    with open('result_file/'+str('last')+'_result'+'/score_dict_'+str(num)+'.pickle', 'rb') as f:
        score_array = pickle.load(f)
    for i in range(len(score_array)):
        if i not in sz_index_dict[num]:
            if score_array[i] > 15:
                tmp=(random.choices(range(0,15),weights=[1,0.3,0.3,0.3,0.03,0.03,0.03,0.03,0.03,0.005,0.005,0.005,0.001,0.01,0.01]))
                score_array[i] = tmp[0]
            else:
                score_array[i] = int(score_array[i]/3)
    score_tmp_dict[num] = score_array      